In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Store Part I results into DataFrame
# Load the csv exported in Part I to a DataFrame
# Create DataFrame from WeatherPy csv 
weather_info = pd.read_csv("output_file.csv")
weather_info

,City,Cloudiness,Country,Humidity,Latitude,Longitude,Max Temperature,Wind Speeds
0,Oruzgan,0,AF,78,32.83,66.00,33.40,4.25
1,Griffith,1,US,54,41.53,-87.42,57.20,13.87
2,Kapaa,75,US,69,22.08,-159.32,82.40,6.93
3,Srostki,95,RU,97,52.42,85.70,28.96,3.33
4,Sitka,90,US,87,57.05,-135.33,51.80,16.11
...,...,...,...,...,...,...,...,...
535,Pemangkat,100,ID,84,1.17,108.97,79.03,3.18
536,Arroyo de la Encomienda,0,ES,93,41.61,-4.80,50.00,5.82
537,Korla,0,CN,73,41.76,86.15,36.75,2.21
538,Buchanan,89,LR,90,5.88,-10.05,77.94,6.20


In [3]:
# Humidity Heatmap
# Configure gmaps.
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
# Set up map
location = weather_info[["Latitude", "Longitude"]].astype(float)
hum = weather_info["Humidity"].astype(float)

fig = gmaps.figure(map_type="SATELLITE")
heat = gmaps.heatmap_layer(location, weights=hum, dissipating=False, max_intensity=100,point_radius=2)

# Add Heatmap layer to map.
fig.add_layer(heat)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Create new DataFrame fitting weather criteria
# Narrow down the cities to fit weather conditions.
# Drop any rows will null values.
perfect = weather_info.loc[(weather_info["Max Temperature"]>= 70)&(weather_info["Max Temperature"]<=80)&(weather_info["Wind Speeds"]<10)&(weather_info["Cloudiness"]==0)]
hotel_df = perfect.dropna()
hotel_df

,City,Cloudiness,Country,Humidity,Latitude,Longitude,Max Temperature,Wind Speeds
55,Keffi,0,NG,88,8.85,7.87,75.20,3.36
63,Rikitea,0,PF,66,-23.12,-134.97,75.96,6.31
143,Sakakah,0,SA,13,29.97,40.21,71.60,3.36
146,Salalah,0,OM,35,17.02,54.09,73.40,4.70
254,Coyhaique,0,CL,33,-45.58,-72.07,71.60,5.82
308,Diffa,0,NE,27,13.32,12.61,77.81,9.73
322,Pandamatenga,0,BW,31,-18.53,25.63,70.90,8.32
381,Rūdsar,0,IR,41,37.14,50.29,71.82,4.97
388,Pāchora,0,IN,53,20.67,75.35,71.69,2.68
414,Vicuña,0,CL,22,-30.03,-70.71,73.00,1.01


In [5]:
# Hotel Map
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""


# Set parameters to search for hotels with 5000 meters.

param = {"radius": 5000,
         "types": "lodging",
         "key": g_key
        }

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    param["location"]=f"{lat},{lng}"
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_name = requests.get(url, params=param).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][3]["name"]
        
    except (KeyError, IndexError):
        print("Skipping")
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.
hotel_df

Skipping
Skipping


,City,Cloudiness,Country,Humidity,Latitude,Longitude,Max Temperature,Wind Speeds,Hotel Name
55,Keffi,0,NG,88,8.85,7.87,75.20,3.36,Otunsha Hotel
63,Rikitea,0,PF,66,-23.12,-134.97,75.96,6.31,
143,Sakakah,0,SA,13,29.97,40.21,71.60,3.36,Twilight for Furnished Apartments
146,Salalah,0,OM,35,17.02,54.09,73.40,4.70,Muscat International Hotel plaza Salalah
254,Coyhaique,0,CL,33,-45.58,-72.07,71.60,5.82,Hotel los Nires
308,Diffa,0,NE,27,13.32,12.61,77.81,9.73,
322,Pandamatenga,0,BW,31,-18.53,25.63,70.90,8.32,Bateleur Farm
381,Rūdsar,0,IR,41,37.14,50.29,71.82,4.97,مشاورین املاک اذین
388,Pāchora,0,IN,53,20.67,75.35,71.69,2.68,Shalikgram rago patil
414,Vicuña,0,CL,22,-30.03,-70.71,73.00,1.01,Parra Hostal


In [6]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info )
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))